Loading of Dataset and Basic Inspections

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('US_orphaned_wells.csv')
df.head()

C:\Users\2924552\AppData\Local\Temp\ipykernel_21464\2010967466.py:5: DtypeWarning: Columns (9,17,20,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('US_orphaned_wells.csv')


,Well identifier,State,County,Well name,Well number,Type,Status,Latitude,Longitude,Prime meridian,...,R dir,Section,Qtr,Qtr_qtr,Qtr_qtr_qtr,Source,Data file date,Well info notes,Location notes,Other notes
0,API:01023198330000,Alabama,Choctaw,A.M. Moseley #2,NaN,OIL,AB,31.870633,-88.262407,NaN,...,W,3.0,NaN,NaN,NaN,State of Alabama Oil and Gas Board,1/18/2022,NaN,Coordinates: USGS converted original NAD27 to ...,Status Date:1974-01-01T00:00:00
1,API:01023199710000,Alabama,Choctaw,Unit 1-5,NaN,OIL,AB,31.868743,-88.243366,NaN,...,W,1.0,NaN,NaN,NaN,State of Alabama Oil and Gas Board,1/18/2022,NaN,Coordinates: USGS converted original NAD27 to ...,Status Date:1974-01-01T00:00:00
2,API:01023199850000,Alabama,Choctaw,John Green #1,NaN,OIL,AB,31.882103,-88.423896,NaN,...,W,31.0,NaN,NaN,NaN,State of Alabama Oil and Gas Board,1/18/2022,NaN,Coordinates: USGS converted original NAD27 to ...,Status Date:1974-01-01T00:00:00
3,API:01039200300000,Alabama,Covington,SMAK-DIXON 31-11 #1,NaN,OIL,AB,31.179635,-86.592117,NaN,...,E,31.0,NaN,NaN,NaN,State of Alabama Oil and Gas Board,1/18/2022,NaN,Coordinates: USGS converted original NAD27 to ...,Status Date:2020-10-01T00:00:00
4,API:01039200310000,Alabama,Covington,SMAK-DIXON 31-10 #1,NaN,OIL,AB,31.179795,-86.589197,NaN,...,E,31.0,NaN,NaN,NaN,State of Alabama Oil and Gas Board,1/18/2022,NaN,Coordinates: USGS converted original NAD27 to ...,Status Date:2020-06-20T00:00:00


Data Inspection

In [2]:
df.info()
df.describe()
df.dtypes

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 117672 entries, 0 to 117671
Data columns (total 23 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Well identifier  117672 non-null  object 
 1   State            117672 non-null  object 
 2   County           117205 non-null  object 
 3   Well name        117646 non-null  object 
 4   Well number      53867 non-null   object 
 5   Type             58090 non-null   object 
 6   Status           111818 non-null  object 
 7   Latitude         117672 non-null  float64
 8   Longitude        117672 non-null  float64
 9   Prime meridian   26628 non-null   object 
 10  Township         47673 non-null   float64
 11  T dir            47673 non-null   object 
 12  Range            47673 non-null   float64
 13  R dir            47673 non-null   object 
 14  Section          47573 non-null   float64
 15  Qtr              23565 non-null   object 
 16  Qtr_qtr          21902 non-null   obje

Well identifier     object
State               object
County              object
Well name           object
Well number         object
Type                object
Status              object
Latitude           float64
Longitude          float64
Prime meridian      object
Township           float64
T dir               object
Range              float64
R dir               object
Section            float64
Qtr                 object
Qtr_qtr             object
Qtr_qtr_qtr         object
Source              object
Data file date      object
Well info notes     object
Location notes      object
Other notes         object
dtype: object

Selection of Columns of Interest - The project will focus on the location, status, type of fluid within those orphaned wells in different states in America


In [3]:
cols_to_keep = ['State','County','Status','Latitude','Longitude']
df = df[cols_to_keep]
df.head()

,State,County,Status,Latitude,Longitude
0,Alabama,Choctaw,AB,31.870633,-88.262407
1,Alabama,Choctaw,AB,31.868743,-88.243366
2,Alabama,Choctaw,AB,31.882103,-88.423896
3,Alabama,Covington,AB,31.179635,-86.592117
4,Alabama,Covington,AB,31.179795,-86.589197


Categorizing Entries in the  "STATUS" Column into few categories - Plugged, Orphaned, Abandoned, Temporary Abandoned, Active, Workover, Cancelled (Dry/Unused), Environmental Reclamation and Unknown

In [4]:
def simplify_status(val):
    val = str(val).strip().upper()

    if val in ['PLUGGED', 'P&A', 'PA', 'PR', 'BOTH PLUGGING AND RECLAMATION', 'XX']:
        return 'PLUGGED'

    elif val in ['ORPHAN', 'ORPHANED', 'ORPHANED WELL', 'OR',
                 'ACT 404 ORPHAN WELL-ENG', 'ACT 404 ORPHAN WELL-INJECTION AND MINING',
                 'POTENTIAL ORPHAN WELL', 'FORFEITED']:
        return 'ORPHANED'

    elif val in ['ABANDONED', 'ABANDONED ORPHANED WELL', 'ABANDONED WELL', 'AB',
                 'ABANDONED, KNOWN LOCATION AND VERIFIED',
                 'ABANDONED, NO EVIDENCE OF EXISTENCE/ UNABLE TO FIND',
                 'ABANDONED, UNKNOWN LOCATION']:
        return 'ABANDONED'

    elif val in ['TA', 'IDLE', 'I', 'SI', 'SUSP', 'S', 'LU']:
        return 'TEMP_ABANDONED'

    elif val in ['ACTIVE', 'OP', 'AL', 'HP', 'IJ']:
        return 'ACTIVE'

    elif val in ['WO']:
        return 'WORKOVER'

    elif val in ['CANCELED', 'DA', 'DM']:
        return 'CANCELLED'

    elif val in ['ENVIRONMENTAL RECLAMATION']:
        return 'RECLAMATION'

    elif val in ['UN', 'UNKNOWN', 'UNKNOWN LOCATED', 'UNKNOWN NOT FOUND']:
        return 'UNKNOWN'

    else:
        return 'OTHER'
df['Status'] = df['Status'].apply(simplify_status)
df.head(100)


,State,County,Status,Latitude,Longitude
0,Alabama,Choctaw,ABANDONED,31.870633,-88.262407
1,Alabama,Choctaw,ABANDONED,31.868743,-88.243366
2,Alabama,Choctaw,ABANDONED,31.882103,-88.423896
3,Alabama,Covington,ABANDONED,31.179635,-86.592117
4,Alabama,Covington,ABANDONED,31.179795,-86.589197
...,...,...,...,...,...
95,Arkansas,COLUMBIA,ABANDONED,33.422700,-93.118300
96,Arkansas,COLUMBIA,ABANDONED,33.423100,-93.120900
97,Arkansas,COLUMBIA,ABANDONED,33.419870,-93.131700
98,Arkansas,COLUMBIA,ABANDONED,33.396060,-93.128040


In [5]:
df.to_csv('cleaned_orphaned_wells.csv', index=False)
